# Lets first install the required packages (for the part 1,2 and 3) and import the libraries

In [4]:
# - Installing packages

#!conda install -c conda-forge folium=0.5.0 --yes # download of folium

#!conda install -c conda-forge beautifulsoup4 --yes # download beautiful soup

#!conda install -c conda-forge geopy --yes # download geopy

#!pip install lxml
#!pip install et_xmlfile
#---------------------------------------------------------------------------------#

#- Importing the libraries
import pandas as pd
import numpy as np
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup as bs
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 


#important resources 


 #1- https://medium.com/analytics-vidhya/web-scraping-wiki-tables-using-beautifulsoup-and-python-6b9ea26d8722 
 #2- https://stackoverflow.com/questions/41720896/beautifulsoup-parsing-html-get-part-of-href
 #3- http://beautiful-soup-4.readthedocs.io/en/latest/


#### To create the above dataframe:

1.The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

2.Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

3.More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

4.If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

5.Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.

6.In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.


## Getting and scrapping the content

In [58]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(url)
soup = bs(source.text, "html.parser")
postal_table = soup.find(class_="wikitable sortable") 

### Into Panda dataframe

In [59]:
#question 1

table_rows = postal_table.find_all('tr')
row_values = []
for tr in table_rows:
    td = tr.find_all('td')
    row_text = [tr.text.strip() for tr in td if tr.text.strip()]
    if row_text:
        row_values.append(row_text)

toronto_data = pd.DataFrame(row_values, columns=["PostalCode", "Borough", "Neighborhood"]) #data frame consisting in PostalCode, Borough, and Neighborhood
toronto_data.head()


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## Working on the data frame 

In [60]:
#Question 2,3,4,5

borough = toronto_data[toronto_data.Borough != 'Not assigned'] # ignoring 'not assigned' 

borough['Neighborhood'].replace('Not assigned', "Borough", inplace=True) #replacing 'Not assigned' neighborhood value with the corresponding Borough value

combined_data = borough.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index(drop=False) # combining neighbours

combined_data.head(15)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [61]:
#question 6
combined_data.shape  #shape

(103, 3)

## Adding coordinates

In [62]:
url = 'http://cocl.us/Geospatial_data' #reading the csv file
coordinates = pd.read_csv(url)

In [63]:
combined_data2 = pd.DataFrame({'PostalCode':combined_data['PostalCode'],  #adding coordinates
                            'Borough':combined_data['Borough'], 
                            'Neighborhood':combined_data['Neighborhood'], 
                            'Latitude':coordinates['Latitude'], 
                            'Longitude':coordinates['Longitude']})

Toronto_df= combined_data2
Toronto_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


### Let's get the geographical coordinates of Toronto.

In [64]:
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

## Vizualizing Toronto and the neighborhoods in it.

In [12]:
# creating a map of toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# adding markers
for lat, lng, borough, neighborhood in zip(Toronto_df['Latitude'], Toronto_df['Longitude'], Toronto_df['Borough'], Toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.5,
        parse_html=False).add_to(map_toronto)
   

In [13]:
map_toronto

#### Define Foursquare Credentials and Version

In [65]:
# foursquare credentials to access their APIs
client_id = 'EZTP0DUNMD3TKQDJDOQFXN3ORJU4FCB5W4RG4DLMVDCGRGFT'
client_secret = 'E12WSECYOLAAVMEKS0KSMWJYG4XUQMFZJQ305DCV4N1VLW1E'
version = '20180605'

print('Your credentails:')
print('CLIENT_ID: ' + client_id)
print('CLIENT_SECRET:' + client_secret)

Your credentails:
CLIENT_ID: EZTP0DUNMD3TKQDJDOQFXN3ORJU4FCB5W4RG4DLMVDCGRGFT
CLIENT_SECRET:E12WSECYOLAAVMEKS0KSMWJYG4XUQMFZJQ305DCV4N1VLW1E


### Exploring the first neighborhood in our dataframe.

In [66]:
Toronto_df.loc[0, 'Neighborhood']

'Malvern, Rouge'

Get the neighborhood's latitude and longitude values.

In [67]:
neighborhood_latitude = Toronto_df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Toronto_df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = Toronto_df.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Malvern, Rouge are 43.806686299999996, -79.19435340000001.


### Now, let's get the top 100 venues that are in the Malvern, Rouge within a radius of 750 meters

#### First, let's create the GET request URL.

In [68]:
LIMIT=100
RADIUS=750

URL='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    client_id, 
    client_secret, 
    version, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    RADIUS, 
    LIMIT)

URL

'https://api.foursquare.com/v2/venues/explore?&client_id=EZTP0DUNMD3TKQDJDOQFXN3ORJU4FCB5W4RG4DLMVDCGRGFT&client_secret=E12WSECYOLAAVMEKS0KSMWJYG4XUQMFZJQ305DCV4N1VLW1E&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=750&limit=100'

Send the GET request and examine the results

In [69]:
results = requests.get(URL).json()

 function that extracts the category of the venue

In [70]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### json in panda dataframe

In [71]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Wendy’s,Fast Food Restaurant,43.807448,-79.199056
1,Wendy's,Fast Food Restaurant,43.802008,-79.198080
2,Staples Morningside,Paper / Office Supplies Store,43.800285,-79.196607
3,Tim Hortons,Coffee Shop,43.802000,-79.198169
4,Lee Valley,Hobby Shop,43.803161,-79.199681
5,Images Salon & Spa,Spa,43.802283,-79.198565
6,Bus Stop: 85 & 116,Bus Station,43.802198,-79.199389
7,Tim Hortons / Esso,Coffee Shop,43.801863,-79.199296
8,Ecopainting inc.,Construction & Landscaping,43.808417,-79.202392
9,Rouge Park - Woodland Trail,Trail,43.801782,-79.200427


 ### Exploring neighborhoods and creating a new dataframe called toronto_venues with Forsquare API

#### creating a function to repeat the same process to all the neighborhoods in Toronto

In [114]:
def getNearbyVenues(names, latitudes, longitudes, radius=250):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            client_id, 
    client_secret, 
    version, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    RADIUS, 
    LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)



#### writing the code to run the above function on each neighborhood and creating a new dataframe called *toronto_venues

In [115]:
toronto_venues = getNearbyVenues(names=Toronto_df['Neighborhood'],
                                 latitudes=Toronto_df['Latitude'],
                                 longitudes=Toronto_df['Longitude'])


Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto, Broadview North (Old East York)
The Danforth West, 

### Lets check the size of the toronto_venues

In [116]:
print(toronto_venues.shape)
toronto_venues.head()

(1030, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Malvern, Rouge",43.806686,-79.194353,Wendy's,43.802008,-79.198080,Fast Food Restaurant
2,"Malvern, Rouge",43.806686,-79.194353,Staples Morningside,43.800285,-79.196607,Paper / Office Supplies Store
3,"Malvern, Rouge",43.806686,-79.194353,Tim Hortons,43.802000,-79.198169,Coffee Shop
4,"Malvern, Rouge",43.806686,-79.194353,Lee Valley,43.803161,-79.199681,Hobby Shop


#### Showing unique categories

In [117]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 8 uniques categories.


##  Analyze Each Neighborhood

In [118]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()



,Neighborhood,Bus Station,Coffee Shop,Construction & Landscaping,Fast Food Restaurant,Hobby Shop,Paper / Office Supplies Store,Spa,Trail
0,"Malvern, Rouge",0,0,0,1,0,0,0,0
1,"Malvern, Rouge",0,0,0,1,0,0,0,0
2,"Malvern, Rouge",0,0,0,0,0,1,0,0
3,"Malvern, Rouge",0,1,0,0,0,0,0,0
4,"Malvern, Rouge",0,0,0,0,1,0,0,0


In [119]:
toronto_onehot.shape

(1030, 9)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [120]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Bus Station,Coffee Shop,Construction & Landscaping,Fast Food Restaurant,Hobby Shop,Paper / Office Supplies Store,Spa,Trail
0,Agincourt,0.1,0.2,0.1,0.2,0.1,0.1,0.1,0.1
1,"Alderwood, Long Branch",0.1,0.2,0.1,0.2,0.1,0.1,0.1,0.1
2,"Bathurst Manor, Wilson Heights, Downsview North",0.1,0.2,0.1,0.2,0.1,0.1,0.1,0.1
3,Bayview Village,0.1,0.2,0.1,0.2,0.1,0.1,0.1,0.1
4,"Bedford Park, Lawrence Manor East",0.1,0.2,0.1,0.2,0.1,0.1,0.1,0.1
...,...,...,...,...,...,...,...,...,...
94,"Willowdale, Willowdale West",0.1,0.2,0.1,0.2,0.1,0.1,0.1,0.1
95,Woburn,0.1,0.2,0.1,0.2,0.1,0.1,0.1,0.1
96,Woodbine Heights,0.1,0.2,0.1,0.2,0.1,0.1,0.1,0.1
97,York Mills West,0.1,0.2,0.1,0.2,0.1,0.1,0.1,0.1


#### Let's print each neighborhood along with the top 5 most common venues

In [121]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                        venue  freq
0                 Coffee Shop   0.2
1        Fast Food Restaurant   0.2
2                 Bus Station   0.1
3  Construction & Landscaping   0.1
4                  Hobby Shop   0.1


----Alderwood, Long Branch----
                        venue  freq
0                 Coffee Shop   0.2
1        Fast Food Restaurant   0.2
2                 Bus Station   0.1
3  Construction & Landscaping   0.1
4                  Hobby Shop   0.1


----Bathurst Manor, Wilson Heights, Downsview North----
                        venue  freq
0                 Coffee Shop   0.2
1        Fast Food Restaurant   0.2
2                 Bus Station   0.1
3  Construction & Landscaping   0.1
4                  Hobby Shop   0.1


----Bayview Village----
                        venue  freq
0                 Coffee Shop   0.2
1        Fast Food Restaurant   0.2
2                 Bus Station   0.1
3  Construction & Landscaping   0.1
4                  Hobby Shop   0.1



#### Let's put that into a *pandas* dataframe

In [122]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 5 venues for each neighborhood.

In [123]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Agincourt,Fast Food Restaurant,Coffee Shop,Trail,Spa,Paper / Office Supplies Store
1,"Alderwood, Long Branch",Fast Food Restaurant,Coffee Shop,Trail,Spa,Paper / Office Supplies Store
2,"Bathurst Manor, Wilson Heights, Downsview North",Fast Food Restaurant,Coffee Shop,Trail,Spa,Paper / Office Supplies Store
3,Bayview Village,Fast Food Restaurant,Coffee Shop,Trail,Spa,Paper / Office Supplies Store
4,"Bedford Park, Lawrence Manor East",Fast Food Restaurant,Coffee Shop,Trail,Spa,Paper / Office Supplies Store


## 4. Cluster Neighborhoods

Run k-means to cluster the neighborhood into 3 clusters

In [124]:
# set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=5).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:120] 

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return_n_iter=True)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 5 venues for each neighborhood.

In [108]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = Toronto_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged =toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,0,Fast Food Restaurant,Coffee Shop,Trail,Spa,Paper / Office Supplies Store
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,0,Fast Food Restaurant,Coffee Shop,Trail,Spa,Paper / Office Supplies Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0,Fast Food Restaurant,Coffee Shop,Trail,Spa,Paper / Office Supplies Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0,Fast Food Restaurant,Coffee Shop,Trail,Spa,Paper / Office Supplies Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,Fast Food Restaurant,Coffee Shop,Trail,Spa,Paper / Office Supplies Store


### Finally, let's visualize the resulting clusters

In [125]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=7,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=2.0).add_to(map_clusters)
map_clusters   




In [103]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Scarborough,0,Fast Food Restaurant,Coffee Shop,Trail,Spa,Paper / Office Supplies Store
1,Scarborough,0,Fast Food Restaurant,Coffee Shop,Trail,Spa,Paper / Office Supplies Store
2,Scarborough,0,Fast Food Restaurant,Coffee Shop,Trail,Spa,Paper / Office Supplies Store
3,Scarborough,0,Fast Food Restaurant,Coffee Shop,Trail,Spa,Paper / Office Supplies Store
4,Scarborough,0,Fast Food Restaurant,Coffee Shop,Trail,Spa,Paper / Office Supplies Store
...,...,...,...,...,...,...,...
98,York,0,Fast Food Restaurant,Coffee Shop,Trail,Spa,Paper / Office Supplies Store
99,Etobicoke,0,Fast Food Restaurant,Coffee Shop,Trail,Spa,Paper / Office Supplies Store
100,Etobicoke,0,Fast Food Restaurant,Coffee Shop,Trail,Spa,Paper / Office Supplies Store
101,Etobicoke,0,Fast Food Restaurant,Coffee Shop,Trail,Spa,Paper / Office Supplies Store
